$\beta = -1$ means that $\beta$ is selected at random in each iteration.

In [102]:
import pandas as pd

def analyze(dataframe: pd.DataFrame, col: int, is_general: bool = False):
    """
    Get averages and #bests of `dataframe` of results from betas experiment.

    `col`: column index of difference for beta = 0 ("d_0.0")
    `is_general`: `True` if `dataframe` is a general table of all instances,
    #bests are counts of minimum averages;
    otherwise #bests are counts of 0.
    """
    diffs = dataframe.iloc[:, col:]
    avgs = diffs.mean()

    other = diffs.min(axis=1) if is_general else 0
    bests = diffs.eq(other, axis=0).sum()

    data = {
        "average": avgs,
        "#bests": bests,
    }

    return pd.DataFrame(data.values(), index=data.keys())

In [103]:
do_save = False
do_display_each = False

In [110]:
from IPython.display import display
from grasp_betas import read_results

BOLD_PROP = "font-weight:bold;"

datalist = []

names = [
    "rd400_267_133",
    "rat783_522_261", # 2
]
for name in names:
    betas = read_results(name)
    
    if do_display_each:
        print(name)

    for p in betas["p"].unique():
        for alpha in betas["alpha"].unique():
            fixed = betas[(betas["p"] == p) & (betas["alpha"] == alpha)]
            concated = pd.concat(
                {
                    "params": fixed.iloc[:, 0:3],
                    "beta": fixed.iloc[:, 3:10],
                    "diff": fixed.iloc[:, 10:],
                },
                axis=1
            )
            
            analysis = analyze(fixed, 10)
            styled_analysis = (
                analysis.style
                    .highlight_min(("average", analysis.columns), axis=1, props=BOLD_PROP)
                    .highlight_max(("#bests", analysis.columns), axis=1, props=BOLD_PROP)
            )
            
            if do_display_each:
                display(concated.style.highlight_min(concated.columns[3:10], axis=1, props=BOLD_PROP))
                display(styled_analysis)

            row = [name, p, alpha]
            # get all averages of this combination
            row.extend(analysis.iloc[0, :])
            datalist.append(row)

params = "instance p alpha".split()
headers = params + analysis.columns.tolist()
table = pd.DataFrame(datalist, columns=headers)

gpdtable = table.groupby(params, sort=False).mean()
display(gpdtable.style.highlight_min(axis=1, props=BOLD_PROP))

analysis = analyze(gpdtable, 0, True)
display(
    analysis.style
        .highlight_min(("average", analysis.columns), axis=1, props=BOLD_PROP)
        .highlight_max(("#bests", analysis.columns), axis=1, props=BOLD_PROP)
)

,d_0.0,d_0.2,d_0.4,d_0.6,d_0.8,d_1.0,d_-1
average,1.790330,1.254365,1.358267,1.120125,1.604385,2.002939,1.466946
#bests,0.000000,2.000000,2.000000,4.000000,3.000000,1.000000,2.000000


In [107]:
# if do_save:
#     highlighted.to_latex(convert_css=True)